In [2]:
import itertools
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.csgraph import shortest_path
import math
from tqdm import tqdm_notebook as tqdm
import pandas as pd

%matplotlib inline

# Epinions

https://snap.stanford.edu/data/soc-Epinions1.html

## データ準備

In [42]:
network = pd.read_table("data/Epinions/soc-Epinions1.txt",header=3)
network.head()

/Users/fuekimasao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,# FromNodeId,ToNodeId
0,0,4
1,0,5
2,0,7
3,0,8
4,0,9


## WeightedCascade

### 各ノードの入次数を計算する

In [43]:
network["ToNodeId"].unique()

array([    4,     5,     7, ..., 75880, 75881, 52098])

In [44]:
node_id = network["ToNodeId"].unique()

In [45]:
in_degree = network.groupby("ToNodeId").size()
in_degree = in_degree.reset_index()
in_degree.columns = ["ToNodeId", "in_degree"]
in_degree.head()

,ToNodeId,in_degree
0,0,636
1,1,802
2,2,237
3,3,40
4,4,125


### 枝確率の計算

枝$e$の終点を頂点$v$  
$d(v)$=頂点$v$の入次数としたとき、
$$ p(e)=\frac{1}{d(v)} $$

In [46]:
in_degree["WC"] = 1/in_degree["in_degree"]
in_degree.head()

,ToNodeId,in_degree,WC
0,0,636,0.001572
1,1,802,0.001247
2,2,237,0.004219
3,3,40,0.025000
4,4,125,0.008000


In [47]:
network_p = pd.merge(network, in_degree, on="ToNodeId", how="left").drop(columns="in_degree")
network_p.head()

,# FromNodeId,ToNodeId,WC
0,0,4,0.008000
1,0,5,0.005682
2,0,7,0.033333
3,0,8,0.009615
4,0,9,0.066667


### データの出力 

In [64]:
network_p.to_csv("data/Epinion/WC.csv",index=False)

## TRIVARENCY

### {0.1,0.01,0.001}をランダムに割り当てる

In [51]:
P = [0.1, 0.01, 0.001]
TR = np.random.choice(P, len(network))
network_p["TR"] = TR

network_p = network_p[["# FromNodeId", "ToNodeId", "TR"]]
network_p.head()

In [65]:
network_p.to_csv("data/Epinion/TR.csv",index=False)

## P=0.1

In [57]:
P = 0.1
network_p["P=0.1"] = P
network_p = network_p[["# FromNodeId", "ToNodeId", "P=0.1"]]
network_p.head()

,# FromNodeId,ToNodeId,P=0.1
0,0,4,0.1
1,0,5,0.1
2,0,7,0.1
3,0,8,0.1
4,0,9,0.1


In [66]:
network_p.to_csv("data/Epinion/P=0_1.csv",index=False)

## P=0.01

In [59]:
P = 0.01
network_p["P=0.01"] = P
network_p = network_p[["# FromNodeId", "ToNodeId", "P=0.01"]]
network_p.head()

,# FromNodeId,ToNodeId,P=0.01
0,0,4,0.01
1,0,5,0.01
2,0,7,0.01
3,0,8,0.01
4,0,9,0.01


In [67]:
network_p.to_csv("data/Epinion/P=0_01.csv",index=False)

## P=0.001

In [61]:
P = 0.001
network_p["P=0.001"] = P
network_p = network_p[["# FromNodeId", "ToNodeId", "P=0.001"]]
network_p.head()

,# FromNodeId,ToNodeId,P=0.001
0,0,4,0.001
1,0,5,0.001
2,0,7,0.001
3,0,8,0.001
4,0,9,0.001


In [68]:
network_p.to_csv("data/Epinion/P=0_001.csv",index=False)

# Pokec

https://snap.stanford.edu/data/soc-Pokec.html

## データ準備

In [71]:
network = pd.read_table("data/Pokec/soc-pokec-relationships.txt",header=-1)
network.head()

/Users/fuekimasao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,0,1
0,1,13
1,1,11
2,1,6
3,1,3
4,1,4


In [75]:
network.columns=["# FromNodeId","ToNodeId"]
network.head()

,# FromNodeId,ToNodeId
0,1,13
1,1,11
2,1,6
3,1,3
4,1,4


## WeightedCascade

### 各ノードの入次数を計算する

In [76]:
network["ToNodeId"].unique()

array([     13,      11,       6, ..., 1632771, 1623923, 1632736])

In [77]:
node_id = network["ToNodeId"].unique()

In [78]:
in_degree = network.groupby("ToNodeId").size()
in_degree = in_degree.reset_index()
in_degree.columns = ["ToNodeId", "in_degree"]
in_degree.head()

,ToNodeId,in_degree
0,1,14
1,2,42
2,3,3
3,4,1
4,5,14


### 枝確率の計算

枝$e$の終点を頂点$v$  
$d(v)$=頂点$v$の入次数としたとき、
$$ p(e)=\frac{1}{d(v)} $$

In [79]:
in_degree["WC"] = 1/in_degree["in_degree"]
in_degree.head()

,ToNodeId,in_degree,WC
0,1,14,0.071429
1,2,42,0.023810
2,3,3,0.333333
3,4,1,1.000000
4,5,14,0.071429


In [85]:
network_p = pd.merge(network, in_degree, on="ToNodeId", how="left").drop(columns="in_degree")
network_p.head()

,# FromNodeId,ToNodeId,WC
0,1,13,0.037037
1,1,11,0.027027
2,1,6,0.071429
3,1,3,0.333333
4,1,4,1.000000


### データの出力 

In [86]:
network_p.to_csv("data/Pokec/WC.csv",index=False)

## TRIVARENCY

### {0.1,0.01,0.001}をランダムに割り当てる

In [87]:
P = [0.1, 0.01, 0.001]
TR = np.random.choice(P, len(network))
network_p["TR"] = TR

network_p = network_p[["# FromNodeId", "ToNodeId", "TR"]]
network_p.head()

,# FromNodeId,ToNodeId,TR
0,1,13,0.010
1,1,11,0.001
2,1,6,0.001
3,1,3,0.010
4,1,4,0.001


In [ ]:
network_p.to_csv("data/Pokec/TR.csv",index=False)

## P=0.1

In [ ]:
P = 0.1
network_p["P=0.1"] = P
network_p = network_p[["# FromNodeId", "ToNodeId", "P=0.1"]]
network_p.head()

In [ ]:
network_p.to_csv("data/Pokec/P=0_1.csv",index=False)

## P=0.01

In [ ]:
P = 0.01
network_p["P=0.01"] = P
network_p = network_p[["# FromNodeId", "ToNodeId", "P=0.01"]]
network_p.head()

In [ ]:
network_p.to_csv("data/Pokec/P=0_01.csv",index=False)

## P=0.001

In [ ]:
P = 0.001
network_p["P=0.001"] = P
network_p = network_p[["# FromNodeId", "ToNodeId", "P=0.001"]]
network_p.head()

In [ ]:
network_p.to_csv("data/Pokec/P=0_001.csv",index=False)